ideja:
- plosna integracije skalarnog i vektorskog polja na proizvoljnoj plohi
- primjeri iz knjige Vektorska analiza - analiticka rjesenja
- numericka rjesenja koristeci MLP
- numericka rjesenja koristeci MLP u noisy situacijama